# L5: Automate Event Planning

## SETUP

In [1]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

In [2]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
# os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"

## TOOLS

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

tool_search = SerperDevTool()
tool_scrape = ScrapeWebsiteTool()

## AGENTS

In [5]:
agent_venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue based on event requirements",
    tools=[tool_search, tool_scrape],
    verbose=True,
    backstory=(
        """with a keen sense of space and understanding of event logistics, you excel at finding and securing the perfect venue that fits the event's theme, size, and budget constraints."""
    ),
)

In [6]:
agent_logistics_manager = Agent(
    role="Logistics Manager",
    goal="Manage all logistics for the event including catering and equipment",
    backstory=(
        """Organized and detail-oriented, you ensure that every logistical aspect of the event from catering to equipment setup is flawlessly executed to create a seamless experience."""
    ),
    verbose=True,
    tools=[tool_search, tool_scrape],
)

In [7]:
agent_marketing_communications = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and communicate with participants",
    backstory=(
        """Create and communicative, you craft compelling messages and engage with potential attendees to maximize event exposure and participation."""
    ),
    verbose=True,
    tools=[tool_search, tool_scrape],
)

## CUSTOM PYDANTIC OBJECT

In [11]:
from pydantic import BaseModel


class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## TASKS

In [13]:
task_venue = Task(
    description="""Find a venue in {event_city} that meets criteria for {event_topic}""",
    expected_output="""All the details of a specifically chosen venue you found to accommodate the event.""",
    human_input=True,  # ask for human feedback
    output_json=VenueDetails,
    output_file="../files/venue_details.json",
    agent=agent_venue_coordinator,
)

In [14]:
task_logistics = Task(
    description="""coordinate catering and equipment for an event with {expected_participants} participants on {tentative_date}.""",
    expected_output="""Confimation of all logistics arrangements including catering and equipment setup.""",
    human_input=True,
    async_execution=True,
    agent=agent_logistics_manager,
)

In [16]:
task_marketing = Task(
    description="""Promote the {event_topic} aiming to engage at least {expected_participants} potential attendees.""",
    expected_output="""Report on marketing activities and attendee engagement formatted as markdown""",
    async_execution=True,
    output_file="../files/marketing_report.md",
    agent=agent_marketing_communications,
)

## CREW

In [18]:
crew_event_management = Crew(
    agents=[
        agent_venue_coordinator,
        agent_logistics_manager,
        agent_marketing_communications,
    ],
    tasks=[task_venue, task_logistics, task_marketing],
    verbose=True,
)

2024-05-27 10:41:33,869 - 8610696192 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [19]:
inputs_event_details = {
    "event_topic": "Tech Innovation Conference",
    "event_description": "A gathering of tech innovators and industry leaders to explore future technologies",
    "event_city": "Louisville, KY",
    "tentative_date": "June 01, 2024",
    "expected_participants": 50,
    "budget": 20000,
    "venue_type": "Convention Center",
}

In [23]:
result = crew_event_management.kickoff(inputs=inputs_event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in Louisville, KY that meets criteria for Tech Innovation Conference


> Entering new CrewAgentExecutor chain...
First, I need to understand what the requirements are for a Tech Innovation Conference. This will likely include a large main hall for keynote speeches, several smaller rooms for breakout sessions, good technological infrastructure (like high-speed internet and audio-visual equipment), and a convenient location for attendees. Let's start by searching for venues in Louisville, KY suitable for hosting a tech conference. 

Action: Search the internet
Action Input: {"search_query": "Tech conference venues in Louisville, KY"}  


Search results: Title: Technology conferences events in Louisville, KY - Eventbrite
Link: https://www.eventbrite.com/d/ky--louisville/technology-conferences/
Snippet: Technology conferences events in Louisville, KY · TALK's 8th Annual Cyber Security Summit · ISSA Kentuc

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 8795 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [24]:
import json
from pprint import pprint as pp

with open("../files/venue_details.json") as f:
    data = json.load(f)

pp(data)

FileNotFoundError: [Errno 2] No such file or directory: '../files/venue_details.json'

In [25]:
from IPython.display import Markdown

Markdown("marketing_report.md")

marketing_report.md